In [4]:
'''Schritt 1
Die Daten aktuellen OSM-Daten werden heruntergeladen und in das richtige Dateiformat gebracht.


Es werden die Daten aus OSM (Download von https://download.geofabrik.de/europe/germany.html)
für die weitere Analyse/Verwendung aufbereitet.
Vorgehen wie hier: https://github.com/vizsim/osm_analyse_strassennamen
bzw. hier: https://journocode.com/de/tutorials/extracting-geodata-from-openstreetmap-with-osmfilter/
beschrieben.
Für Linux: apt install osmctools
Für Windows: osmfilter und osmconvert runterladen
außerdem ogr2ogr besorgen.
@author: lukas
'''

'Schritt 1\nDie Daten aktuellen OSM-Daten werden heruntergeladen und in das richtige Dateiformat gebracht.\n\n\nEs werden die Daten aus OSM (Download von https://download.geofabrik.de/europe/germany.html)\nfür die weitere Analyse/Verwendung aufbereitet.\nVorgehen wie hier: https://github.com/vizsim/osm_analyse_strassennamen\nbzw. hier: https://journocode.com/de/tutorials/extracting-geodata-from-openstreetmap-with-osmfilter/\nbeschrieben.\nFür Linux: apt install osmctools\nFür Windows: osmfilter und osmconvert runterladen\naußerdem ogr2ogr besorgen.\n@author: lukas\n'

In [8]:
import subprocess   #schnittstelle zum Betriebssystem, soll statt 'os' verwendet werden.
import os           #schnittstelle zum Betriebssystem, osmfilter hat mit subprocess nicht funktioniert
import geopandas
import ogr2ogr
import requests
from datetime import date

datum=date.today().strftime("%y%m%d")
datum = "231114"

print('used date: ' + str(datum) + '\nimport finished')

used date: 231114
import finished


In [2]:
#----------brandenburg mit Berlin--------

#Herunterladen der aktuellen OSM-Daten. Angeblich bei großen Dateien besser, wenn gestückelt...

url = 'https://download.geofabrik.de/europe/germany/brandenburg-latest.osm.pbf'
r = requests.get(url, allow_redirects=True, stream = True)

with open("../OSM-roh/brandenburg-latest_"+datum+".osm.pbf","wb") as f:
    for chunk in r.iter_content(chunk_size=1024):
  
         if chunk:
             f.write(chunk)

In [ ]:
##-----------germany--------------------

#Herunterladen der aktuellen OSM-Daten. Angeblich bei großen Dateien besser, wenn gestückelt...


url = 'https://download.geofabrik.de/europe/germany-latest.osm.pbf'
r = requests.get(url, allow_redirects=True, stream = True)

with open("../OSM-roh/germany-latest_"+datum+".osm.pbf","wb") as f:
    for chunk in r.iter_content(chunk_size=1024):
  
         if chunk:
             f.write(chunk)

In [5]:
"""Nimmt die erste angegebene Datei und konvertiert sie. Output (-o=) ist die zweite angegebene Datei
Berechnung mit Dateiformat 05m soll schneller sein."""

#Windows
#subprocess.run(['osmconvert64', '../OSM-roh/brandenburg-latest_'+datum+'.osm.pbf', '-o=../OSM-roh/brandenburg-latest_'+datum+'.osm.o5m'])
#Linux
##----Brandenburg mit Berlin
#subprocess.run(['osmconvert', '../OSM-roh/brandenburg-latest_'+datum+'.osm.pbf', '-o=../OSM-roh/brandenburg-latest_'+datum+'.osm.o5m'])  
subprocess.run(['osmconvert', '../OSM-roh/230315/brandenburg-latest_230315.osm.pbf', '-o=../OSM-roh/230315/brandenburg-latest_230315.osm.o5m'])  

##----germany
#subprocess.run(['osmconvert', '../OSM-roh/germany-latest_'+datum+'.osm.pbf', '-o=../OSM-roh/germany-latest_'+datum+'.osm.o5m'])  

CompletedProcess(args=['osmconvert', '../OSM-roh/230315/brandenburg-latest_230315.osm.pbf', '-o=../OSM-roh/230315/brandenburg-latest_230315.osm.o5m'], returncode=0)

In [19]:
"""aus gesamten OSM-Daten werden alle Straßen und Wege sefiltert und in neue DAtei geschrieben.
Neue Datei im Format osm, damit im nächsten Schritt lesbar.
Hinzugenommen zum zu_Nah-Prpojekt: =service =living_street =*_link =trunk
Entfernt im Vergleich zum zu_Nah-Projekt: =cycleway =footway =pedestrian
"""
selection = "highway=*ary* =unclassified =residential =road =track =path =service =living_street =trunk =trunk_link"  # | bicycle=yes"


##----Brandenburg mit Berlin
#os.system('osmfilter ../OSM-roh/brandenburg-latest_'+datum+'.osm.o5m --keep="'+selection+'" -o=../OSM-roh/drivable_infra_Brandenburg_'+datum+'.osm')
#os.system('osmfilter ../OSM-roh/230315/brandenburg-latest_230315.osm.o5m --keep="'+selection+'" -o=../OSM-roh/230315/drivable_infra_Brandenburg_230315.osm')

##----germany
#os.system('osmfilter ../OSM-roh/231114de/germany-latest_'+datum+'.osm.o5m --keep='+selection+' -o=../OSM-roh/231114de/drivable_infra_Germany_'+datum+'.osm')
os.system('osmfilter ../OSM-roh/231114de/germany-latest_'+datum+'.osm.o5m --keep="'+selection+'" -o=../OSM-roh/231114de/drivable_infra_Germany_test'+datum+'.osm')


2

In [20]:
"""Wandelt osm in gpkg um -> soll besser sein für Handhabung mit geopandas.
Dabei werden die Attribute, die in der Ini-Datei stehen, als eigene Spalten angelegt.
In .ini-Datei hinzugenommen zum zu_Nah-Prpojekt: 
cycleway:separation:left,cycleway:separation:right,
separation:left,separation:left
cycleway:bicycle,cycleway:both:bicycle,cycleway:right:bicycle,cycleway:left:bicycle,
bicycle_road,
oneway,
cycleway:width,cycleway:both:width,cycleway:right:width,cycleway:left:width
cycleway:right:marking:left, cycleway:right:marking:right, cycleway:left:marking:left, cycleway:left:marking:right, cycleway:right:marking:both,cycleway:left:marking:both,cycleway:both:marking:left,cycleway:both:marking:right,cycleway:both:marking:both,
cycleway:right:traffic_sign, cycleway:left:traffic_sign, cycleway:both:traffic_sign,
parking,parking:both,parking:left,parking:right,
parking:both:orientation,parking:left:orientation,parking:right:orientation,
cycleway:right:separation:right,cycleway:left:separation:right,cycleway:both:separation:right,cycleway:right:separation:left,cycleway:left:separation:left,cycleway:both:separation:left,cycleway:right:separation:both,cycleway:left:separation:both,cycleway:both:separation:both,
cycleway:right:buffer:right,cycleway:left:buffer:right,cycleway:both:buffer:right,cycleway:right:buffer:left,cycleway:left:buffer:left,cycleway:both:buffer:left,cycleway:right:buffer:both,cycleway:left:buffer:both,cycleway:both:buffer:both

"""
##----Brandenburg mit Berlin
#ogr2ogr.main(["-overwrite","--config", "OSM_CONFIG_FILE","osmconf_drivable.ini","-skipfailures","-f", "GPKG", '../OSM-roh/drivable_infra_Brandenburg_'+datum+'.gpkg','../OSM-roh/drivable_infra_Brandenburg_'+datum+'.osm'])

#ogr2ogr.main(["-overwrite","--config", "OSM_CONFIG_FILE","osmconf_drivable.ini","-skipfailures","-f", "GPKG", '../OSM-roh/230315/drivable_infra_Brandenburg_230315.gpkg','../OSM-roh/230315/drivable_infra_Brandenburg_230315.osm'])


##----germany
ogr2ogr.main(["-overwrite","--config", "OSM_CONFIG_FILE","osmconf_drivable.ini","-skipfailures","-f", "GPKG", '../OSM-roh/231114de/drivable_infra_Germany_'+datum+'.gpkg','../OSM-roh/231114de/drivable_infra_Germany_'+datum+'.osm'])



True